In [1]:
# for each ligand, generate 5 protein-ligand descriptors
# 1 protein is the matching protein, 4 are random proteins
# descriptors are saved as a .npy file

# also outputs a csv file, containing
# lig_id, pro_id, descriptor path, score
# score = 1 if lig_id & pro_id matches, 0 otherwise

In [2]:
from htmd.ui import *
from htmd.molecule.voxeldescriptors import *
from pathlib import Path
import numpy as np
import pickle as pk
from multiprocessing import Pool
import pandas as pd

%load_ext autoreload
%autoreload 2

/home/alex/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


ffevaluate module is in beta version

Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. 
https://dx.doi.org/10.1021/acs.jctc.6b00049
Documentation: http://software.acellera.com/
To update: conda update htmd -c acellera -c psi4

You are on the latest HTMD version (1.13.9).



In [3]:
np.random.seed = 1
num_random_proteins = 4

In [4]:
ligand_df = pd.read_csv('./data/csv/ligand_data.csv', index_col=False)

In [5]:
# split ligands here
train_size = int(len(ligand_df) * 0.90)

train_df = ligand_df.iloc[:train_size]
test_df = ligand_df.iloc[train_size:]

In [6]:
def gen_data(df, base_dest, csv_dest):
    Path(base_dest).mkdir(exist_ok=True, parents=True)
    
    all_lig_idxs = []
    all_protein_idxs = []
    dests = []
    score = []

    def gen_pro_lig_voxel_pairs(lig_idx, protein_idxs, path):
        with open(path, 'rb') as f:
            lig_features, centers = pk.load(f)

        for pidx in protein_idxs:
            mol = Molecule('./data/training_data/'+str(pidx).zfill(4)+'_pro_cg.pdb', keepaltloc='all')
            pro_features, _ = getVoxelDescriptors(mol, usercenters=np.array(centers), voxelsize=1, method='NUMBA')
            pro_features = pro_features.reshape(24, 24, 24, pro_features.shape[1])
            combined = np.concatenate((pro_features, lig_features), axis=3)
            combined = combined.astype(np.float32)

            dest = base_dest + '/' + str(pidx).zfill(4) + '_pro_' + str(lig_idx).zfill(4) + '_lig.npy'
            np.save(dest, combined)

            all_lig_idxs.append(lig_idx)
            all_protein_idxs.append(pidx)
            dests.append(dest)
            score.append(1 if pidx == lig_idx else 0)
        
    for row in df.itertuples():
        df_idx = row[0]
        lig_idx = row[1]
        path = row[2]
        
        # generate 4 random proteins to pair
        r = range(0,len(df)-1)
        selection = np.random.choice(r, num_random_proteins, replace=False)
        selection = [len(df)-1 if s == df_idx else s for s in selection]
        # convert them into protein idxs
        protein_idxs = list(df.iloc[selection].id.values)
        protein_idxs.append(lig_idx)
        
        gen_pro_lig_voxel_pairs(lig_idx, protein_idxs, path)
        
    pro_lig_record = pd.DataFrame({'lig_id': all_lig_idxs, 'pro_id': all_protein_idxs, 'dests': dests, 'score': score}, index=None)
    pro_lig_record.to_csv(csv_dest, index=None)

In [7]:
train_dest = './processed_data/pro_lig_voxels/train'
test_dest = './processed_data/pro_lig_voxels/test'

In [8]:
gen_data(test_df, test_dest, 'test_lig_pro_pairs.csv')
gen_data(train_df, train_dest, 'train_lig_pro_pairs.csv')

/home/alex/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \
